In [210]:
from agents import *
from notebook import psource
import random
import time

## CONTENTS

* Implementation for 3.1 
* Implementation for 3.2
* Implementation for 3.3

# Q3.1 

# THINGS in our world

This space includes possible chair(s), trolley(s) and one Person

In [211]:
class Chair(Thing):
    pass

class Trolley(Thing):
    pass

class Person(Thing):
    pass


# ENVIRONMENT - Public Space
Here, we will create and update the environment based on interactions with the agent and things

In [227]:
class Space(GraphicEnvironment):
    
    #Called this function just for the delay so I can read my outputs comfortably
    def run(self, steps=1000, delay=1.75):
        """Run the Environment for given number of time steps,
        but update the GUI too."""
        for step in range(steps):
            self.update(delay)
            if self.is_done():
                break
            self.step()
        self.update(delay)
        
    #This function looks for things in the agent's location and nearby
    def percept(self, agent):
        '''return a list of things that are in our agent's location'''
        things = self.list_things_at(agent.location)
        
        # Find things in all neighbouring directions of the agent
        # Which is nodes in Top Left, Up, Top Right, Left, Right, Bottom Left, Down and Bottom Right
        print(agent.location)
        x= agent.location[0]
        y=agent.location[1]
        thingsTl = self.list_things_at([x-1,y-1])
        thingsU = self.list_things_at([x,y-1])
        thingsTr = self.list_things_at([x+1,y-1])
        thingsL = self.list_things_at([x-1,y])
        thingsR = self.list_things_at([x+1,y])
        thingsBl = self.list_things_at([x-1,y+1])
        thingsD = self.list_things_at([x,y+1])
        thingsBr = self.list_things_at([x+1,y+1])
        
        
        print("Top left: ", self.list_things_at([x-1,y-1]))
        print("Up: ", self.list_things_at([x,y-1]))
        print("Top right: ", self.list_things_at([x+1,y-1]))
        print("Left: ", self.list_things_at([x-1,y]))
        print("Right: ", self.list_things_at([x+1,y]))
        print("Bottom left: ", self.list_things_at([x-1,y+1]))
        print("Down:", self.list_things_at([x,y+1]))
        print("Bottom Right:", self.list_things_at([x+1,y+1]))
        
        
        #Locations of the cells to the Left, Right, Top and Bottom of the agent
        leftForward = [agent.location[0]-1,agent.location[1]]
        rightForward = [agent.location[0]+1, agent.location[1]]
        bottomForward = [agent.location[0], agent.location[1]+1]
        topForward = [agent.location[0], agent.location[1]-1]
        
        
        loc = copy.deepcopy(agent.location) # find out the target location
        #Check if agent is about to bump into a wall or a human in all 4 directions
        #Human is checked for by comparing the 4 neighbouring cells of the robot
        
        if agent.direction.direction == Direction.R:
            
            if rightForward == humanLocation:
                print("Oops, person on the right tile, WILL TURN!")
                things.append(Bump())
            
            if agent.location[0] < n-1:
                    loc[0] += 1
            else:
                print("Out of bounds on the right, WILL TURN!")
                things.append(Bump())
                
        elif agent.direction.direction == Direction.L:
            
            if leftForward == humanLocation:
                print("Oops, person on the left tile, WILL TURN!")
                things.append(Bump())
  
            if agent.location[0] > 0:
                loc[0] -= 1
            else:
                print("Out of bounds on the left, WILL TURN!")
                things.append(Bump())
        
        elif agent.direction.direction == Direction.D:
            
            if bottomForward == humanLocation:
                print("Oops, person on the bottom tile, WILL TURN!")
                things.append(Bump())
            
            if agent.location[1] < n-1:
                    loc[1] += 1
            else:
                print("Out of bounds at the bottom, WILL TURN!")
                things.append(Bump())
                
        elif agent.direction.direction == Direction.U:
            if topForward == humanLocation:
                    print("Oops, person on the top tile, WILL TURN!")
                    things.append(Bump())
            
            if agent.location[1] > 0:
                loc[1] -= 1
            else:
                print("Out of bounds at the top, WILL TURN!")
                things.append(Bump())
        elif loc == humanLocation:
            print("I collided with human")
            things.append(Bump())
            
        else:
            print("Agent is not in any NEWS directions")
        print("ALL THINGS: ",things)
        
        #This returns its output to the DisinfectingRobot.program(percepts)
        return things, thingsTl, thingsU, thingsTr, thingsL, thingsR, thingsBl, thingsD, thingsBr
    
    def execute_action(self, agent, action):
        '''changes the state of the environment based on what the agent does.'''
        
        if action == 'turnright':
            print('{} decided to {} at location: {}'.format(str(agent)[1:-1], action, agent.location))
            agent.turn(Direction.R)
        elif action == 'turnleft':
            print('{} decided to {} at location: {}'.format(str(agent)[1:-1], action, agent.location))
            agent.turn(Direction.L)
        elif action == 'moveforward':
            print('{} decided to move {}wards at location: {}'.format(str(agent)[1:-1], agent.direction.direction, agent.location))
            agent.moveforward()
        elif action == "cleanChair":
            # Check for chair at agent's location and if true clean it
            items = self.list_things_at(agent.location, tclass=Chair)
            if len(items) != 0:
                if agent.clean(items[0]):
                    print('{} clean {} at location: {}'
                          .format(str(agent)[1:-1], str(items[0])[1:-1], agent.location))
                    self.delete_thing(items[0])
            
            print("Current THINGS = ", len(self.things))
            print("Initial THINGS = ", initial_items)
            time.sleep(3)
            #Once cleaned, make the robot go to Starting Point (0,0)
            agent.direction.direction = Direction.U
            y = agent.location[1]
            for moveup in range(y):
                agent.moveforward()
            
            agent.direction.direction = Direction.L
            x = agent.location[0]
            for moveleft in range(x):
                agent.moveforward()
                    
        elif action == "cleanTrolley":
            # Check for chair at agent's location and if true clean it
            items = self.list_things_at(agent.location, tclass=Trolley)
            if len(items) != 0:
                if agent.clean(items[0]):
                    print('{} clean {} at location: {}'
                          .format(str(agent)[1:-1], str(items[0])[1:-1], agent.location))
                    self.delete_thing(items[0])
            
            print("Current THINGS = ", len(self.things))
            print("Initial THINGS = ", initial_items)
            
            #Once cleaned, make the robot go to Starting Point (0,0)
            agent.direction.direction = Direction.U
            y = agent.location[1]
            for moveup in range(y):
                agent.moveforward()
            
            agent.direction.direction = Direction.L
            x = agent.location[0]
            print("Going back to STARTING POINT using ",y," steps UPWARDS and ",x,"steps LEFTWARDS.")
            for moveleft in range(x):
                agent.moveforward()
                    
    def is_done(self):
        '''The program cannot continue without any agents or cleanable surfaces or 
           when an item (Chair or Trolley) is cleaned, so it's ended. '''
        no_cleanables = not any(isinstance(thing, Chair) or isinstance(thing, Trolley) for thing in self.things)
        dead_agents = not any(agent.is_alive() for agent in self.agents)
        cleaned = False
        #Compare total no. of items at the start to total no. of items in the current state
        if (len(self.things) < initial_items):
            cleaned = True
            return cleaned
        return dead_agents or no_cleanables 


# AGENT - COVID Disinfecting Robot
Here, we will code the robot's actions and its percepts (when it sees something in the neighbouring cells)

In [213]:
from random import choice

class DisinfectingRobot(Agent):
    location = [0,0]
    direction = Direction("down")
    
    def moveforward(self, success=True):
        '''moveforward possible only if success (i.e. valid destination location)'''
        if not success:
            return
        if self.direction.direction == Direction.R:
            self.location[0] += 1
        elif self.direction.direction == Direction.L:
            self.location[0] -= 1
        elif self.direction.direction == Direction.D:
            self.location[1] += 1
        elif self.direction.direction == Direction.U:
            self.location[1] -= 1
    
    def turn(self, d):
        self.direction = self.direction + d
        
    def clean(self, thing):
        '''returns True upon success or False otherwise'''
        if isinstance(thing, Chair):
            return True
        elif isinstance(thing, Trolley):
            return True
        return False
    
        
def program(percepts):
    '''Returns an action based on it's percepts'''
    print("ALL PERCEPTS:", percepts)
    
    #Remember we returned all of the following values in Space.percept(agent). 
    #Now we will decide what to do when we see items.
    things, thingsTl, thingsU, thingsTr, thingsL, thingsR, thingsBl, thingsD, thingsBr = percepts
    for p in things: # Clean only Chair or Trolley
        if isinstance(p, Chair):
            return 'cleanChair'
        elif isinstance(p, Trolley):
            return 'cleanTrolley'
        
        if isinstance(p,Bump): # If at the boundary or besides a Person then turn
            turn = False
            choice = random.choice((1,2));
        else:
            choice = random.choice((1,2,3,4)) # 1-right, 2-left, others-forward
        
    if choice == 1:
        return 'turnright'
    elif choice == 2:
        return 'turnleft'
    else:
        return 'moveforward'
    

In [230]:
#Since, its an nxn grid we only need the length of one edge
n = int(input("Input the size of the edge: "))

#Initialize floor, robot, and human. 
floor = Space(n,n, color={'DisinfectingRobot': (200,0,0), 'Chair': (0, 200, 200), 'Trolley': (230, 115, 40), 'Person': (0,200,0)})
robot = DisinfectingRobot(program)
human = Person()

#Add robot to our environment so it can interact with it
floor.add_thing(robot, [0,0])

# m<n, as mentioned in the question
maxM = n-1

# m can also be anything within that limit
frequency = random.randint(0,maxM)

# So, create 2 objects: Chair and Trolley, m times (so they are equal)
for x in range(frequency):
    
    #Only add a chair to a non occupied location
    while True:
        chair_location = [random.randint(0,n-1), random.randint(0,n-1)]
        if len(floor.list_things_at(chair_location)) != 0:
            #Chair's tile is occupied!
            continue
        else:
            floor.add_thing(Chair(), chair_location)
            break
            
    #Only add a trolley ir to a non occupied location
    while True:
        trolley_location = [random.randint(0,n-1), random.randint(0,n-1)]
        if len(floor.list_things_at(trolley_location)) != 0:
            #Trolley's tile is occupied!
            continue
        else:
            floor.add_thing(Trolley(), trolley_location)
            break

#Also add 1 Person to a non-occupied location
while True:
    person_location = [random.randint(0,n-1), random.randint(0,n-1)]
    if len(floor.list_things_at(person_location)) != 0:
        #Tile is occupied!
        continue
    else:
        #humanLocation is used by agent to avoid the human
        humanLocation = person_location
        floor.add_thing(human, person_location)
        break

#Keep track of the initial things (Objects) we created
#In all program 2 objects are certain: The Robot and the Human.
#We need to know how many Chairs and Trolleys were created, can do it by: 2*frequency
initial_items = 2+(2*frequency)
time.sleep(2)

print("COVID Disinfecting Robot started at [0,0], facing down. Let's see if its found any chair or trolley!")
floor.run()


,
,


# Q3.2

### NEW ROBOT THAT CAN SEE NEARBY AND CLEAN ALL ITEMS
The robot will go back to starting point (0,0) once it cleans an object and restart it's cleaning again

In [238]:
class CleanAllSpace(GraphicEnvironment):
    
    #Called this function just for the delay so I can read my outputs comfortably
    def run(self, steps=1000, delay=0.25):
        """Run the Environment for given number of time steps,
        but update the GUI too."""
        for step in range(steps):
            self.update(delay)
            if self.is_done():
                break
            self.step()
        self.update(delay)
        
    #This function looks for things in the agent's location and nearby
    def percept(self, agent):
        '''return a list of things that are in our agent's location'''
        things = self.list_things_at(agent.location)
        
        # Find things in all neighbouring directions of the agent
        # Which is nodes in Top Left, Up, Top Right, Left, Right, Bottom Left, Down and Bottom Right
        print(agent.location)
        x= agent.location[0]
        y=agent.location[1]
        thingsTl = self.list_things_at([x-1,y-1])
        thingsU = self.list_things_at([x,y-1])
        thingsTr = self.list_things_at([x+1,y-1])
        thingsL = self.list_things_at([x-1,y])
        thingsR = self.list_things_at([x+1,y])
        thingsBl = self.list_things_at([x-1,y+1])
        thingsD = self.list_things_at([x,y+1])
        thingsBr = self.list_things_at([x+1,y+1])
        
        
        print("Top left: ", self.list_things_at([x-1,y-1]))
        print("Up: ", self.list_things_at([x,y-1]))
        print("Top right: ", self.list_things_at([x+1,y-1]))
        print("Left: ", self.list_things_at([x-1,y]))
        print("Right: ", self.list_things_at([x+1,y]))
        print("Bottom left: ", self.list_things_at([x-1,y+1]))
        print("Down:", self.list_things_at([x,y+1]))
        print("Bottom Right:", self.list_things_at([x+1,y+1]))
        
        
        #Locations of the cells to the Left, Right, Top and Bottom of the agent
        leftForward = [agent.location[0]-1,agent.location[1]]
        rightForward = [agent.location[0]+1, agent.location[1]]
        bottomForward = [agent.location[0], agent.location[1]+1]
        topForward = [agent.location[0], agent.location[1]-1]
        
        
        loc = copy.deepcopy(agent.location) # find out the target location
        #Check if agent is about to bump into a wall or a human in all 4 directions
        #Human is checked for by comparing the 4 neighbouring cells of the robot
        
        if agent.direction.direction == Direction.R:
            
            if rightForward == humanLocation:
                print("Oops, person on the right tile, WILL TURN!")
                things.append(Bump())
            
            if agent.location[0] < n-1:
                    loc[0] += 1
            else:
                print("Out of bounds on the right, WILL TURN!")
                things.append(Bump())
                
        elif agent.direction.direction == Direction.L:
            
            if leftForward == humanLocation:
                print("Oops, person on the left tile, WILL TURN!")
                things.append(Bump())
  
            if agent.location[0] > 0:
                loc[0] -= 1
            else:
                print("Out of bounds on the left, WILL TURN!")
                things.append(Bump())
        
        elif agent.direction.direction == Direction.D:
            
            if bottomForward == humanLocation:
                print("Oops, person on the bottom tile, WILL TURN!")
                things.append(Bump())
            
            if agent.location[1] < n-1:
                    loc[1] += 1
            else:
                print("Out of bounds at the bottom, WILL TURN!")
                things.append(Bump())
                
        elif agent.direction.direction == Direction.U:
            if topForward == humanLocation:
                    print("Oops, person on the top tile, WILL TURN!")
                    things.append(Bump())
            
            if agent.location[1] > 0:
                loc[1] -= 1
            else:
                print("Out of bounds at the top, WILL TURN!")
                things.append(Bump())
        elif loc == humanLocation:
            print("I collided with human")
            things.append(Bump())
            
        else:
            print("Agent is not in any NEWS directions")
        print("ALL THINGS: ",things)
        
        #This returns its output to the DisinfectingRobot.program(percepts)
        return things, thingsTl, thingsU, thingsTr, thingsL, thingsR, thingsBl, thingsD, thingsBr
    
    def execute_action(self, agent, action):
        '''changes the state of the environment based on what the agent does.'''
        
        if action == 'turnright':
            print('{} decided to {} at location: {}'.format(str(agent)[1:-1], action, agent.location))
            agent.turn(Direction.R)
        elif action == 'turnleft':
            print('{} decided to {} at location: {}'.format(str(agent)[1:-1], action, agent.location))
            agent.turn(Direction.L)
        elif action == 'moveforward':
            print('{} decided to move {}wards at location: {}'.format(str(agent)[1:-1], agent.direction.direction, agent.location))
            agent.moveforward()
        elif action == "cleanChair":
            # Check for chair at agent's location and if true clean it
            items = self.list_things_at(agent.location, tclass=Chair)
            if len(items) != 0:
                if agent.clean(items[0]):
                    print('{} clean {} at location: {}'
                          .format(str(agent)[1:-1], str(items[0])[1:-1], agent.location))
                    self.delete_thing(items[0])
            
            #Once cleaned, make the robot go to Starting Point (0,0)
            agent.direction.direction = Direction.U
            y = agent.location[1]
            for moveup in range(y):
                agent.moveforward()
            
            agent.direction.direction = Direction.L
            x = agent.location[0]
            for moveleft in range(x):
                agent.moveforward()
                    
        elif action == "cleanTrolley":
            # Check for chair at agent's location and if true clean it
            items = self.list_things_at(agent.location, tclass=Trolley)
            if len(items) != 0:
                if agent.clean(items[0]):
                    print('{} clean {} at location: {}'
                          .format(str(agent)[1:-1], str(items[0])[1:-1], agent.location))
                    self.delete_thing(items[0])
            
            #Once cleaned, make the robot go to Starting Point (0,0)
            agent.direction.direction = Direction.U
            y = agent.location[1]
            for moveup in range(y):
                agent.moveforward()
            
            agent.direction.direction = Direction.L
            x = agent.location[0]
            print("Going back to STARTING POINT using ",y," steps UPWARDS and ",x,"steps LEFTWARDS.")
            for moveleft in range(x):
                agent.moveforward()
                    
    def is_done(self):
        '''The program cannot continue without any agents or cleanable surfaces or 
           when an item (Chair or Trolley) is cleaned, so it's ended. '''
        no_cleanables = not any(isinstance(thing, Chair) or isinstance(thing, Trolley) for thing in self.things)
        dead_agents = not any(agent.is_alive() for agent in self.agents)
        cleaned = False
        #Compare total no. of items at the start to total no. of items in the current state
        if (len(self.things) == 2):
            cleaned = True
            print("All cleanable surfaces have been cleaned! END OF SIMULATION.")
            time.sleep(3)
            return cleaned
        return dead_agents or no_cleanables 


In [236]:
from random import choice

class DisinfectingRobot(Agent):
    location = [0,0]
    direction = Direction("down")
    
    def moveforward(self, success=True):
        '''moveforward possible only if success (i.e. valid destination location)'''
        if not success:
            return
        if self.direction.direction == Direction.R:
            self.location[0] += 1
        elif self.direction.direction == Direction.L:
            self.location[0] -= 1
        elif self.direction.direction == Direction.D:
            self.location[1] += 1
        elif self.direction.direction == Direction.U:
            self.location[1] -= 1
    
    def turn(self, d):
        self.direction = self.direction + d
        
    def clean(self, thing):
        '''returns True upon success or False otherwise'''
        if isinstance(thing, Chair):
            return True
        elif isinstance(thing, Trolley):
            return True
        return False
    
        
def program(percepts):
    '''Returns an action based on it's percepts'''
    print("ALL PERCEPTS:", percepts)
    
    #Remember we returned all of the following values in Space.percept(agent). 
    #Now we will decide what to do when we see items.
    things, thingsTl, thingsU, thingsTr, thingsL, thingsR, thingsBl, thingsD, thingsBr = percepts
    for p in things: # Clean only Chair or Trolley
        if isinstance(p, Chair):
            return 'cleanChair'
        elif isinstance(p, Trolley):
            return 'cleanTrolley'
        
        if isinstance(p,Bump): # If at the boundary or besides a Person then turn
            turn = False
            choice = random.choice((1,2));
        else:
            choice = random.choice((1,2,3,4)) # 1-right, 2-left, others-forward
        
    if choice == 1:
        return 'turnright'
    elif choice == 2:
        return 'turnleft'
    else:
        return 'moveforward'
    

In [237]:
#Since, its an nxn grid we only need the length of one edge
n = int(input("Input the size of the edge: "))

#Initialize floor, robot, and human. 
floor = CleanAllSpace(n,n, color={'DisinfectingRobot': (200,0,0), 'Chair': (0, 200, 200), 'Trolley': (230, 115, 40), 'Person': (0,200,0)})
robot = DisinfectingRobot(program)
human = Person()

#Add robot to our environment so it can interact with it
floor.add_thing(robot, [0,0])

# m<n, as mentioned in the question
maxM = n-1

# m can also be anything within that limit
frequency = random.randint(0,maxM)

# So, create 2 objects: Chair and Trolley, m times (so they are equal)
for x in range(frequency):
    
    #Only add a chair to a non occupied location
    while True:
        chair_location = [random.randint(0,n-1), random.randint(0,n-1)]
        if len(floor.list_things_at(chair_location)) != 0:
            #Chair's tile is occupied!
            continue
        else:
            floor.add_thing(Chair(), chair_location)
            break
            
    #Only add a trolley ir to a non occupied location
    while True:
        trolley_location = [random.randint(0,n-1), random.randint(0,n-1)]
        if len(floor.list_things_at(trolley_location)) != 0:
            #Trolley's tile is occupied!
            continue
        else:
            floor.add_thing(Trolley(), trolley_location)
            break

#Also add 1 Person to a non-occupied location
while True:
    person_location = [random.randint(0,n-1), random.randint(0,n-1)]
    if len(floor.list_things_at(person_location)) != 0:
        #Tile is occupied!
        continue
    else:
        #humanLocation is used by agent to avoid the human
        humanLocation = person_location
        floor.add_thing(human, person_location)
        break

#Keep track of the initial things (Objects) we created
#In all program 2 objects are certain: The Robot and the Human.
#We need to know how many Chairs and Trolleys were created, can do it by: 2*frequency
initial_items = 2+(2*frequency)
time.sleep(2)

print("COVID Disinfecting Robot started at [0,0], facing down. Let's see if its found any chair or trolley!")
floor.run()


,,
,,
,,


# Q3.3

### ROBOT CLEANS ALL OBJECT IN 1 GO 
The robot won't come back to origin but go and clean all objects then come back

In [244]:
class OneGoCleanAllSpace(GraphicEnvironment):
    
    #Called this function just for the delay so I can read my outputs comfortably
    def run(self, steps=1000, delay=0.25):
        """Run the Environment for given number of time steps,
        but update the GUI too."""
        for step in range(steps):
            self.update(delay)
            if self.is_done():
                break
            self.step()
        self.update(delay)
        
    #This function looks for things in the agent's location and nearby
    def percept(self, agent):
        '''return a list of things that are in our agent's location'''
        things = self.list_things_at(agent.location)
        
        # Find things in all neighbouring directions of the agent
        # Which is nodes in Top Left, Up, Top Right, Left, Right, Bottom Left, Down and Bottom Right
        print(agent.location)
        x= agent.location[0]
        y=agent.location[1]
        thingsTl = self.list_things_at([x-1,y-1])
        thingsU = self.list_things_at([x,y-1])
        thingsTr = self.list_things_at([x+1,y-1])
        thingsL = self.list_things_at([x-1,y])
        thingsR = self.list_things_at([x+1,y])
        thingsBl = self.list_things_at([x-1,y+1])
        thingsD = self.list_things_at([x,y+1])
        thingsBr = self.list_things_at([x+1,y+1])
        
        
        print("Top left: ", self.list_things_at([x-1,y-1]))
        print("Up: ", self.list_things_at([x,y-1]))
        print("Top right: ", self.list_things_at([x+1,y-1]))
        print("Left: ", self.list_things_at([x-1,y]))
        print("Right: ", self.list_things_at([x+1,y]))
        print("Bottom left: ", self.list_things_at([x-1,y+1]))
        print("Down:", self.list_things_at([x,y+1]))
        print("Bottom Right:", self.list_things_at([x+1,y+1]))
        
        
        #Locations of the cells to the Left, Right, Top and Bottom of the agent
        leftForward = [agent.location[0]-1,agent.location[1]]
        rightForward = [agent.location[0]+1, agent.location[1]]
        bottomForward = [agent.location[0], agent.location[1]+1]
        topForward = [agent.location[0], agent.location[1]-1]
        
        
        loc = copy.deepcopy(agent.location) # find out the target location
        #Check if agent is about to bump into a wall or a human in all 4 directions
        #Human is checked for by comparing the 4 neighbouring cells of the robot
        
        if agent.direction.direction == Direction.R:
            
            if rightForward == humanLocation:
                print("Oops, person on the right tile, WILL TURN!")
                things.append(Bump())
            
            if agent.location[0] < n-1:
                    loc[0] += 1
            else:
                print("Out of bounds on the right, WILL TURN!")
                things.append(Bump())
                
        elif agent.direction.direction == Direction.L:
            
            if leftForward == humanLocation:
                print("Oops, person on the left tile, WILL TURN!")
                things.append(Bump())
  
            if agent.location[0] > 0:
                loc[0] -= 1
            else:
                print("Out of bounds on the left, WILL TURN!")
                things.append(Bump())
        
        elif agent.direction.direction == Direction.D:
            
            if bottomForward == humanLocation:
                print("Oops, person on the bottom tile, WILL TURN!")
                things.append(Bump())
            
            if agent.location[1] < n-1:
                    loc[1] += 1
            else:
                print("Out of bounds at the bottom, WILL TURN!")
                things.append(Bump())
                
        elif agent.direction.direction == Direction.U:
            if topForward == humanLocation:
                    print("Oops, person on the top tile, WILL TURN!")
                    things.append(Bump())
            
            if agent.location[1] > 0:
                loc[1] -= 1
            else:
                print("Out of bounds at the top, WILL TURN!")
                things.append(Bump())
        elif loc == humanLocation:
            print("I collided with human")
            things.append(Bump())
            
        else:
            print("Agent is not in any NEWS directions")
        print("ALL THINGS: ",things)
        
        #This returns its output to the DisinfectingRobot.program(percepts)
        return things, thingsTl, thingsU, thingsTr, thingsL, thingsR, thingsBl, thingsD, thingsBr
    
    def execute_action(self, agent, action):
        '''changes the state of the environment based on what the agent does.'''
        
        if action == 'turnright':
            print('{} decided to {} at location: {}'.format(str(agent)[1:-1], action, agent.location))
            agent.turn(Direction.R)
        elif action == 'turnleft':
            print('{} decided to {} at location: {}'.format(str(agent)[1:-1], action, agent.location))
            agent.turn(Direction.L)
        elif action == 'moveforward':
            print('{} decided to move {}wards at location: {}'.format(str(agent)[1:-1], agent.direction.direction, agent.location))
            agent.moveforward()
        elif action == "cleanChair":
            # Check for chair at agent's location and if true clean it
            items = self.list_things_at(agent.location, tclass=Chair)
            if len(items) != 0:
                if agent.clean(items[0]):
                    print('{} clean {} at location: {}'
                          .format(str(agent)[1:-1], str(items[0])[1:-1], agent.location))
                    self.delete_thing(items[0])
            
            #In Q3.3, the robot is meant to continue cleaning all surfaces and then go to origin
            #Once all surfaces are cleaned, make the robot go to Starting Point (0,0)
            if len(self.things) == 2:
                agent.direction.direction = Direction.U
                y = agent.location[1]
                for moveup in range(y):
                    agent.moveforward()

                agent.direction.direction = Direction.L
                x = agent.location[0]
                for moveleft in range(x):
                    agent.moveforward()

                    
        elif action == "cleanTrolley":
            # Check for chair at agent's location and if true clean it
            items = self.list_things_at(agent.location, tclass=Trolley)
            if len(items) != 0:
                if agent.clean(items[0]):
                    print('{} clean {} at location: {}'
                          .format(str(agent)[1:-1], str(items[0])[1:-1], agent.location))
                    self.delete_thing(items[0])
            
            #Once all surfaces are cleaned, make the robot go to Starting Point (0,0)
            if len(self.things) == 2:
                agent.direction.direction = Direction.U
                y = agent.location[1]
                for moveup in range(y):
                    agent.moveforward()

                agent.direction.direction = Direction.L
                x = agent.location[0]
                for moveleft in range(x):
                    agent.moveforward()
            
                    
    def is_done(self):
        '''The program cannot continue without any agents or cleanable surfaces or 
           when an item (Chair or Trolley) is cleaned, so it's ended. '''
        no_cleanables = not any(isinstance(thing, Chair) or isinstance(thing, Trolley) for thing in self.things)
        dead_agents = not any(agent.is_alive() for agent in self.agents)
        cleaned = False
        #Compare total no. of items at the start to total no. of items in the current state
        if (len(self.things) == 2):
            cleaned = True
            print("All cleanable surfaces have been cleaned! END OF SIMULATION.")
            time.sleep(3)
            return cleaned
        return dead_agents or no_cleanables 


In [245]:
from random import choice

class DisinfectingRobot(Agent):
    location = [0,0]
    direction = Direction("down")
    
    def moveforward(self, success=True):
        '''moveforward possible only if success (i.e. valid destination location)'''
        if not success:
            return
        if self.direction.direction == Direction.R:
            self.location[0] += 1
        elif self.direction.direction == Direction.L:
            self.location[0] -= 1
        elif self.direction.direction == Direction.D:
            self.location[1] += 1
        elif self.direction.direction == Direction.U:
            self.location[1] -= 1
    
    def turn(self, d):
        self.direction = self.direction + d
        
    def clean(self, thing):
        '''returns True upon success or False otherwise'''
        if isinstance(thing, Chair):
            return True
        elif isinstance(thing, Trolley):
            return True
        return False
    
        
def program(percepts):
    '''Returns an action based on it's percepts'''
    print("ALL PERCEPTS:", percepts)
    
    #Remember we returned all of the following values in Space.percept(agent). 
    #Now we will decide what to do when we see items.
    things, thingsTl, thingsU, thingsTr, thingsL, thingsR, thingsBl, thingsD, thingsBr = percepts
    for p in things: # Clean only Chair or Trolley
        if isinstance(p, Chair):
            return 'cleanChair'
        elif isinstance(p, Trolley):
            return 'cleanTrolley'
        
        if isinstance(p,Bump): # If at the boundary or besides a Person then turn
            turn = False
            choice = random.choice((1,2));
        else:
            choice = random.choice((1,2,3,4)) # 1-right, 2-left, others-forward
        
    if choice == 1:
        return 'turnright'
    elif choice == 2:
        return 'turnleft'
    else:
        return 'moveforward'
    

In [247]:
#Since, its an nxn grid we only need the length of one edge
n = int(input("Input the size of the edge: "))

#Initialize floor, robot, and human. 
floor = OneGoCleanAllSpace(n,n, color={'DisinfectingRobot': (200,0,0), 'Chair': (0, 200, 200), 'Trolley': (230, 115, 40), 'Person': (0,200,0)})
robot = DisinfectingRobot(program)
human = Person()

#Add robot to our environment so it can interact with it
floor.add_thing(robot, [0,0])

# m<n, as mentioned in the question
maxM = n-1

# m can also be anything within that limit
frequency = random.randint(0,maxM)

# So, create 2 objects: Chair and Trolley, m times (so they are equal)
for x in range(frequency):
    
    #Only add a chair to a non occupied location
    while True:
        chair_location = [random.randint(0,n-1), random.randint(0,n-1)]
        if len(floor.list_things_at(chair_location)) != 0:
            #Chair's tile is occupied!
            continue
        else:
            floor.add_thing(Chair(), chair_location)
            break
            
    #Only add a trolley ir to a non occupied location
    while True:
        trolley_location = [random.randint(0,n-1), random.randint(0,n-1)]
        if len(floor.list_things_at(trolley_location)) != 0:
            #Trolley's tile is occupied!
            continue
        else:
            floor.add_thing(Trolley(), trolley_location)
            break

#Also add 1 Person to a non-occupied location
while True:
    person_location = [random.randint(0,n-1), random.randint(0,n-1)]
    if len(floor.list_things_at(person_location)) != 0:
        #Tile is occupied!
        continue
    else:
        #humanLocation is used by agent to avoid the human
        humanLocation = person_location
        floor.add_thing(human, person_location)
        break

#Keep track of the initial things (Objects) we created
#In all program 2 objects are certain: The Robot and the Human.
#We need to know how many Chairs and Trolleys were created, can do it by: 2*frequency
initial_items = 2+(2*frequency)
time.sleep(2)

print("COVID Disinfecting Robot started at [0,0], facing down. Let's see if its found any chair or trolley!")
floor.run()


,,
,,
,,
